In [9]:
import torch

from PoincareMaps.data import prepare_data, compute_rfa
from PoincareMaps.model import PoincareEmbedding, PoincareDistance
from torch.utils.data import TensorDataset, DataLoader

In [2]:
dset = "ToggleSwitch"
root = "root"

features, labels = prepare_data(f"/home/roh3635/hyperbolic-cancer/PoincareMaps/datasets/{dset}", with_labels=True, normalize=False, n_pca=0)

In [3]:
rfa = compute_rfa(
    features,
    mode="features", 
    k_neighbours=15, 
    distlocal="minkowski", 
    distfn="MFIsym", 
    connected=True, 
    sigma=1.0
) # Pairwise distances in the original data space

Computing laplacian...
Laplacian computed in 0.01 sec
Computing RFA...
RFA computed in 0.02 sec


In [10]:
device = "cpu"
indices = torch.arange(len(rfa))

indices = indices.to(device)
rfa = rfa.to(device)

dataset = TensorDataset(indices, rfa)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [6]:
predictor = PoincareEmbedding(
    len(dataset), 
    2,
    dist=PoincareDistance,
    max_norm=1,
    Qdist="laplace", 
    lossfn="klSym",
    gamma=1.0,
    cuda=0
)
# predictor.size = dataset size
# predictor.lt = embedding matrix (inputs are indices)
# predictor.dist = distance function
# predictor.lossfn = loss function
# predictor.Qdist = distribution of the Poincaré ball
# predictor.gamma = gamma (temperature)

In [15]:
batch = next(iter(dataloader))
outputs = predictor(batch[0]) # [batch_size, len(dataset)]

assert outputs.sum(dim=-1).allclose(torch.ones(len(batch[0])))